In [2]:
import torch
import torch.optim
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
# 按照《Pytorch学习（三）定义自己的数据集及加载训练》：https://blog.csdn.net/sinat_42239797/article/details/90641659#commentBox
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
#读取数据
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.__version__

'1.1.0'

In [3]:
import torch
pthfile = 'abcd.pth'
net = torch.load(pthfile)
print(net)

OrderedDict([('layers1.0.weight', tensor([[[[ 1.8257e-01,  1.5317e-02,  1.3582e-01, -1.1202e-02,  1.6871e-01],
          [-4.6739e-02,  1.2932e-01,  1.8934e-01,  4.3195e-02,  4.3009e-02],
          [ 1.8067e-01, -8.4124e-02,  1.7186e-01,  1.4487e-02, -1.6903e-01],
          [ 1.6952e-01, -6.6686e-03, -3.7434e-02,  1.5012e-01,  8.2020e-02],
          [-4.1564e-02,  1.6412e-01,  1.6452e-01,  1.6233e-02, -1.9747e-01]]],


        [[[-7.2019e-02, -1.2435e-01, -1.9226e-01,  1.5919e-01,  3.9503e-02],
          [-1.7289e-01, -6.0031e-02, -8.5985e-02, -3.7858e-02, -5.3231e-02],
          [ 1.1106e-01, -1.1401e-01,  1.2929e-01, -1.2699e-01,  1.5169e-01],
          [ 1.2231e-01, -7.9796e-02,  1.2611e-01, -6.2268e-02, -1.6979e-01],
          [ 1.3851e-01, -6.9646e-02,  1.3625e-01,  2.6241e-02, -8.3104e-02]]],


        [[[ 8.0832e-02,  5.8467e-02,  1.7999e-01,  1.1539e-02,  1.6371e-01],
          [ 1.3161e-01, -7.5371e-02, -3.1425e-02, -6.3906e-02,  5.8752e-02],
          [-5.2019e-02,  1.1301e-0

for i in net:
    print i 
    print type(net[i])
    print "aa:",net[i].data.size()
    print "bb:",net[i].requires_grad
    break
///////output://////

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-3-cc9b6f5bdaca> in <module>()
----> 1 for i in net:
      2     print i
      3     print type(net[i])
      4     print "aa:",net[i].data.size()
      5     print "bb:",net[i].requires_grad

TypeError: 'SqueezeNet' object is not iterable

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layers1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=5,stride=1,padding=2),#1代表1个通道 32代表kernel数，kernelsize代表大小
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layers2 = nn.Sequential(
            nn.Conv2d(32,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layers3 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(20*15*64,300),
            nn.ReLU(inplace=True),
            nn.Linear(300,5),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        x = self.layers1(x)
        x = self.layers2(x)
        x = self.layers3(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [5]:
import torchvision.models as models
pthfile = 'abcd.pth'
cnn = CNN()
cnn.load_state_dict(torch.load(pthfile))
print(cnn)

CNN(
  (layers1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layers2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layers3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=19200, out_features=300, bias=True)
    (1): ReLU(inplace)
    (2): Lin

In [6]:
def default_loader(path):
        return Image.open(path)# 如果是我们的depth map 图片格式是L——灰度图。
    
class MyDataset(Dataset): 
#创建自己的类： MyDataset,这个类是继承的torch.utils.data.Dataset
     def __init__(self,x,y, transform=None,target_transform=None, loader=default_loader):
        super(MyDataset,self).__init__()#对继承自父类的属性进行初始化
        self.x = x
        self.y = y
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        #*************************** #使用__getitem__()对数据进行预处理并返回想要的信息**********************
     def __getitem__(self, index):#这个方法是必须要有的，用于按照索引读取每个元素的具体内容
            fn, label = self.x[index],self.y[index]
            img = self.loader(root+'depth2/'+fn)# 按照路径读取图片
            if self.transform is not None:
                img = self.transform(img)#数据标签转换为Tensor
            return img,label#return回哪些内容，那么我们在训练时循环读取每个batch时，就能获得哪些内容
     def __len__(self):#这个函数也必须要写，它返回的是数据集的长度，也就是多少张图片，要和loader的长度作区分
            return len(self.x)

In [7]:
#*************************************数据集的设置************************************
root =os.getcwd()+ '/lmdb_source_data/'#数据集的地址
print(root)
#先建立一个总的list-->numpy型的，然后
txt = root+'depth2_list2.txt'#depth2_list2，有1100个数据 depth2_list，有1104个数据
x,y=[],[]
fh = open(txt,'r')
for line in fh: #迭代该列表#按行循环txt文本中的内
    line = line.strip('\n')
    line = line.rstrip('\n')# 删除 本行string 字符串末尾的指定字符，这个方法的详细介绍自己查询python
    words = line.split()#用split将该行分割成列表  split的默认参数是空格，所以不传递任何参数时分割空格
    x.append(words[0])
    y.append(int(words[1]))
    #把txt里的内容读入imgs列表保存，具体是words几要看txt内容而定 
    # 很显然，根据我刚才截图所示txt的内容，words[0]是图片信息，words[1]是lable  

/home/asber/Documents/RobotLearning/lmdb_source_data/


In [8]:
data=MyDataset(x=x,y=y, transform=transforms.ToTensor())
data_loader = DataLoader(dataset=data,\
                         batch_size=1,shuffle=True,num_workers=1)
print len(data)

1100


In [23]:
import cv2
img = cv2.imread('./lmdb_source_data/depth2/depth2242025.jpg',-1)#结果为1
cv2.imshow("fuck",img)
cv2.waitKey(3)

-1

In [24]:
cv2.destroyAllWindows()

In [25]:
# pytorch的tensor，Image，numpy和opencv四种格式的相互转换
# https://blog.csdn.net/weixin_42662358/article/details/90448566

In [26]:
img = torch.from_numpy(img) 
img = img.view(1,1,160,120)
out = cnn(img.float())
print out

tensor([[1.5903, 3.9486, 2.1725, 0.9913, 0.0000]], grad_fn=<ReluBackward1>)


In [19]:
for i ,(img,label) in enumerate(data_loader):
    #img = img.view(10,1,160,120)
    img = Variable(img)
    out = cnn(img)
    print out,out.sum()
    pre = torch.max(out.data,1)[1]
    print label,pre

tensor([[1.2072, 1.8779, 1.3377, 0.9846, 0.4008]], grad_fn=<ReluBackward1>) tensor(5.8082, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.5350, 2.2347, 3.2251, 1.3952, 0.2549]], grad_fn=<ReluBackward1>) tensor(7.6450, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[1.0761, 2.4931, 1.6927, 0.1553, 0.3781]], grad_fn=<ReluBackward1>) tensor(5.7953, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.8043, 2.9508, 2.7062, 0.2405, 0.2515]], grad_fn=<ReluBackward1>) tensor(6.9533, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.7117, 2.7465, 3.7319, 1.2267, 0.6759]], grad_fn=<ReluBackward1>) tensor(9.0928, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.1427, 2.6833, 3.8411, 1.9896, 0.7181]], grad_fn=<ReluBackward1>) tensor(9.3747, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[0.6868, 1.2844, 3.8271, 2.3278, 0.3897]], grad_fn=<ReluBackward1>) tensor(8.5158, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.9757, 2.6837, 1.

tensor([[1.0156, 2.8275, 1.5986, 0.3935, 0.4824]], grad_fn=<ReluBackward1>) tensor(6.3177, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.4445, 2.1674, 2.9282, 2.5720, 0.2938]], grad_fn=<ReluBackward1>) tensor(8.4060, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.2381, 1.6046, 0.5775, 1.2533, 0.6400]], grad_fn=<ReluBackward1>) tensor(4.3135, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.6924, 1.7999, 3.8120, 1.4192, 0.5790]], grad_fn=<ReluBackward1>) tensor(8.3025, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.9435, 2.7595, 2.3503, 0.4683, 0.6167]], grad_fn=<ReluBackward1>) tensor(7.1382, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.7988, 2.3881, 0.9436, 0.5864, 0.4109]], grad_fn=<ReluBackward1>) tensor(5.1278, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.5664, 1.9158, 3.3179, 1.2318, 0.6594]], grad_fn=<ReluBackward1>) tensor(7.6912, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.7495, 2.6797, 3.

tensor([0]) tensor([1])
tensor([[0.5580, 2.1526, 3.3109, 1.9351, 0.8929]], grad_fn=<ReluBackward1>) tensor(8.8495, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[0.7870, 3.0666, 1.2259, 0.4742, 0.3130]], grad_fn=<ReluBackward1>) tensor(5.8666, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.2814, 1.4781, 3.7412, 2.8748, 1.0480]], grad_fn=<ReluBackward1>) tensor(9.4235, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.5851, 1.6544, 2.6926, 1.9507, 0.4488]], grad_fn=<ReluBackward1>) tensor(7.3315, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.7654, 2.3777, 4.0206, 1.7517, 0.5704]], grad_fn=<ReluBackward1>) tensor(9.4858, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[1.4153, 2.1571, 1.3123, 0.6928, 0.2087]], grad_fn=<ReluBackward1>) tensor(5.7862, grad_fn=<SumBackward0>)
tensor([3]) tensor([1])
tensor([[0.2421, 1.8532, 3.3405, 2.1146, 0.3228]], grad_fn=<ReluBackward1>) tensor(7.8732, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
ten

tensor([[1.1346, 2.6759, 1.9973, 0.0000, 0.2162]], grad_fn=<ReluBackward1>) tensor(6.0239, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.9117, 2.7380, 2.2176, 1.4335, 0.7106]], grad_fn=<ReluBackward1>) tensor(8.0114, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.7822, 2.7365, 2.5746, 1.1348, 0.7845]], grad_fn=<ReluBackward1>) tensor(8.0125, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[1.1268, 2.7503, 2.9250, 1.3425, 0.4376]], grad_fn=<ReluBackward1>) tensor(8.5822, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.6718, 3.2233, 3.2122, 1.0329, 0.4887]], grad_fn=<ReluBackward1>) tensor(8.6290, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.6685, 2.5839, 2.9334, 1.7018, 0.3674]], grad_fn=<ReluBackward1>) tensor(8.2550, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[1.1640, 1.1649, 0.6225, 0.7258, 0.7003]], grad_fn=<ReluBackward1>) tensor(4.3776, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.8709, 2.6002, 2.

tensor([[0.4685, 1.6721, 3.4457, 1.6086, 0.4566]], grad_fn=<ReluBackward1>) tensor(7.6516, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.4887, 1.3822, 1.2596, 1.3180, 1.1418]], grad_fn=<ReluBackward1>) tensor(5.5903, grad_fn=<SumBackward0>)
tensor([4]) tensor([1])
tensor([[0.6104, 1.7930, 3.7377, 1.5729, 0.8806]], grad_fn=<ReluBackward1>) tensor(8.5945, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.7298, 1.9522, 1.8346, 1.3177, 0.8312]], grad_fn=<ReluBackward1>) tensor(6.6655, grad_fn=<SumBackward0>)
tensor([3]) tensor([1])
tensor([[0.8095, 2.2327, 3.4711, 1.3331, 0.6006]], grad_fn=<ReluBackward1>) tensor(8.4469, grad_fn=<SumBackward0>)
tensor([0]) tensor([2])
tensor([[0.7304, 1.3651, 3.7171, 2.3225, 0.4643]], grad_fn=<ReluBackward1>) tensor(8.5995, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.8732, 3.2623, 2.5102, 0.4575, 0.4286]], grad_fn=<ReluBackward1>) tensor(7.5318, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.8727, 2.9109, 2.

...

tensor([[0.4448, 2.1039, 3.3442, 1.9977, 0.3243]], grad_fn=<ReluBackward1>) tensor(8.2149, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[0.4299, 2.6223, 3.2547, 1.7058, 0.4158]], grad_fn=<ReluBackward1>) tensor(8.4285, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[1.3913, 1.9091, 1.2521, 1.0149, 0.5831]], grad_fn=<ReluBackward1>) tensor(6.1506, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.7978, 2.7445, 2.1998, 0.3074, 0.2555]], grad_fn=<ReluBackward1>) tensor(6.3049, grad_fn=<SumBackward0>)
tensor([2]) tensor([1])
tensor([[0.8411, 2.3687, 3.2606, 1.2325, 0.3876]], grad_fn=<ReluBackward1>) tensor(8.0904, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[0.5647, 0.5337, 2.4719, 2.3155, 0.5978]], grad_fn=<ReluBackward1>) tensor(6.4834, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.6815, 2.3768, 3.6485, 1.5248, 0.8624]], grad_fn=<ReluBackward1>) tensor(9.0939, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.9295, 2.7688, 3.

tensor([[0.8365, 1.1809, 2.6005, 1.5288, 0.6612]], grad_fn=<ReluBackward1>) tensor(6.8080, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.9014, 2.4082, 3.7362, 2.1164, 0.3483]], grad_fn=<ReluBackward1>) tensor(9.5105, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.3115, 2.4806, 3.5509, 1.3100, 0.4142]], grad_fn=<ReluBackward1>) tensor(8.0672, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.7136, 1.4453, 2.5937, 1.7017, 0.5141]], grad_fn=<ReluBackward1>) tensor(6.9685, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[1.3793, 2.9866, 1.4719, 0.3558, 0.4400]], grad_fn=<ReluBackward1>) tensor(6.6337, grad_fn=<SumBackward0>)
tensor([2]) tensor([1])
tensor([[1.1608, 2.0760, 1.4886, 0.2983, 0.7322]], grad_fn=<ReluBackward1>) tensor(5.7560, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[1.3593, 1.6795, 1.0185, 0.6712, 0.8928]], grad_fn=<ReluBackward1>) tensor(5.6214, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[1.4936, 2.0114, 0.

tensor([[0.8069, 1.2620, 1.6198, 1.1342, 1.0514]], grad_fn=<ReluBackward1>) tensor(5.8743, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.4648, 1.1782, 2.8486, 2.1372, 0.5541]], grad_fn=<ReluBackward1>) tensor(7.1829, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.6220, 2.9411, 1.4458, 0.0000, 0.0142]], grad_fn=<ReluBackward1>) tensor(5.0232, grad_fn=<SumBackward0>)
tensor([2]) tensor([1])
tensor([[0.8392, 3.0601, 2.5234, 0.3539, 0.3825]], grad_fn=<ReluBackward1>) tensor(7.1592, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.4964, 0.6246, 2.6497, 2.1207, 0.7040]], grad_fn=<ReluBackward1>) tensor(6.5953, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.6464, 1.3460, 2.2885, 2.3910, 0.2804]], grad_fn=<ReluBackward1>) tensor(6.9523, grad_fn=<SumBackward0>)
tensor([3]) tensor([3])
tensor([[0.5849, 1.3935, 0.9267, 1.1031, 0.9789]], grad_fn=<ReluBackward1>) tensor(4.9870, grad_fn=<SumBackward0>)
tensor([4]) tensor([1])
tensor([[0.9295, 2.0059, 2.

tensor([[0.5230, 2.2175, 3.0338, 1.0710, 0.7489]], grad_fn=<ReluBackward1>) tensor(7.5942, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.7201, 3.2632, 3.5654, 1.3940, 0.3154]], grad_fn=<ReluBackward1>) tensor(9.2581, grad_fn=<SumBackward0>)
tensor([1]) tensor([2])
tensor([[0.6722, 2.1092, 3.4006, 1.5950, 0.6546]], grad_fn=<ReluBackward1>) tensor(8.4317, grad_fn=<SumBackward0>)
tensor([3]) tensor([2])
tensor([[1.1431, 2.2569, 1.7565, 0.6839, 0.9056]], grad_fn=<ReluBackward1>) tensor(6.7460, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.6209, 1.3459, 1.2577, 0.9628, 1.0314]], grad_fn=<ReluBackward1>) tensor(5.2188, grad_fn=<SumBackward0>)
tensor([4]) tensor([1])
tensor([[1.2885, 1.8364, 0.9169, 0.3501, 0.9128]], grad_fn=<ReluBackward1>) tensor(5.3048, grad_fn=<SumBackward0>)
tensor([0]) tensor([1])
tensor([[0.7608, 2.0610, 0.4273, 0.9404, 0.7007]], grad_fn=<ReluBackward1>) tensor(4.8901, grad_fn=<SumBackward0>)
tensor([3]) tensor([1])
tensor([[0.5179, 3.4964, 3.

tensor([[1.3991, 2.2409, 1.0286, 0.9929, 0.6175]], grad_fn=<ReluBackward1>) tensor(6.2791, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[0.6494, 1.9431, 1.8314, 1.5074, 0.6847]], grad_fn=<ReluBackward1>) tensor(6.6160, grad_fn=<SumBackward0>)
tensor([3]) tensor([1])
tensor([[0.8013, 1.4478, 2.5349, 1.2884, 0.5905]], grad_fn=<ReluBackward1>) tensor(6.6629, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.9186, 2.9601, 2.5465, 1.0855, 0.7099]], grad_fn=<ReluBackward1>) tensor(8.2207, grad_fn=<SumBackward0>)
tensor([1]) tensor([1])
tensor([[1.2100, 2.1200, 1.7141, 0.9159, 0.4294]], grad_fn=<ReluBackward1>) tensor(6.3894, grad_fn=<SumBackward0>)
tensor([2]) tensor([1])
tensor([[0.4186, 1.8966, 2.4397, 2.0694, 0.4895]], grad_fn=<ReluBackward1>) tensor(7.3137, grad_fn=<SumBackward0>)
tensor([2]) tensor([2])
tensor([[0.4565, 1.8031, 1.7190, 0.9463, 1.0440]], grad_fn=<ReluBackward1>) tensor(5.9688, grad_fn=<SumBackward0>)
tensor([3]) tensor([1])
tensor([[0.6672, 1.0465, 0.

In [20]:
# 首先，我的网络没有dropout，其次，我的输入没有归一化，第三输出没有softmax，导致out.sum()每次都不一样
# 文中使用的方向输出五个离散的角速度，然后按照输出分权给每个角速度。
# 看paper里说，机器人现在只能通过很面前的障碍物决定避开，而人是可以预见碰撞，更好的避开的（也可以说是人的脑子是有一个模型的）
# 其次，我们看使用传统的CNN的方法，我们的数据采用是时序的，虽然打乱了，但是每个tag的样本数都不一样多，这也会造成一定的偏差。

---#上面是对AI在robot中application的想法
# 现在论文看两遍，只看到  turning-full-right（0）  turning-half-right(1) go-straightforward（2） turning-half-left(3)
# turning-full-left(4)
